In [4]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -U ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.5 MB/s eta 0:00:00


In [3]:
!ls /content/drive/MyDrive/OD_project/my_car_dataset


train


In [ ]:
from ultralytics import YOLO
import os

# 1. 모델 로드 (YOLOv11n - 가장 가벼운 모델)
# 이전 실험(m)과 비교하기 위해 n 모델을 사용합니다.
model = YOLO('yolo11n.pt')

# 2. 데이터셋 경로
DATA_YAML_PATH = '/content/drive/MyDrive/OD_project/my_car_dataset/yolo_final_data/data.yaml'

# 3. T4 GPU 최적화 & 속도 중심 학습 설정
results = model.train(
    data=DATA_YAML_PATH,
    epochs=100,
    patience=30,

    # --- Nano 모델 최적화 세팅 ---
    batch=32,             # L4에서 n 모델은 128~256도 가능합니다. 학습 속도가 매우 빨라집니다.
    imgsz=512,             # 640 유지는 정확도-속도 밸런스에 좋습니다.
    workers=2,             # 데이터 로딩 병목 방지

    # --- 학습 알고리즘 ---
    optimizer='AdamW',
    lr0=0.0015,             # 배치 사이즈가 커졌으므로 학습률을 약간 상향 조정
    cos_lr=True,           # 부드러운 수렴을 위해 사용

    # --- 시스템 설정 ---
    device=0,
    project='SafeDrive_Speed_Exp',
    name='yolo11n_fast_v1',
    amp=True,              # L4의 Tensor Core를 활용한 연산 가속 (필수)
    pretrained=True,
    exist_ok=True,
)

print("🚀 YOLOv11n 학습이 완료되었습니다!")

In [ ]:
import glob
import yaml
import os

DATASET_ROOT = "/content/drive/MyDrive/datasets/my_dataset"

label_files = glob.glob(f"{DATASET_ROOT}/labels/train/*.txt")

class_ids = set()

for lf in label_files:
    with open(lf, "r") as f:
        for line in f:
            if line.strip():
                class_ids.add(int(line.split()[0]))

nc = max(class_ids) + 1

names = [f"class_{i}" for i in range(nc)]

data_yaml = {
    "path": DATASET_ROOT,
    "train": "images/train",
    "val": "images/val",
    "nc": nc,
    "names": names
}

yaml_path = os.path.join(DATASET_ROOT, "data.yaml")

with open(yaml_path, "w") as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print(f"✅ data.yaml 자동 생성 완료 → {yaml_path}")
print("nc =", nc)
print("names =", names)


In [ ]:
model.train(
    data="/content/drive/MyDrive/OD_project/yolo_csv_split/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    device=0,
    project="yolov11_train",
    name="exp_yolov11m"
)
